Orain arte egin ditugun inguratze guztiak gris eskalan egin dira eta hemen egingo duguna berriz hsv eskalan egingo da. HSV sistema erabiliko dugu koloreen filtrazioak egiteko egokiagoa delako, baita errazena berarekin lan egiteko ere. 

__HSV__

1. 0tik-360ra kolorea izango da, baina OpenCV 0tik-180ra.
    1. 0 gorria izango da.
    2. 60 horia izango da.
    3. 120 berdea izango da.
    4. 180 cian izango da.
    5. 240 urdina izango da.
    6. 300 morea izango da.
2. 0tik-255era saturazioa izango da.
    1. 0 saturaziorik gabe, txuria.
    2. 255 kolore maximoa izango da.
3. 0tik-255era argitasuna izango da.
    1. 0 beltza izango da.
    2. 255 kolorea argitasun guztiarekin.

In [1]:
import numpy as np
import cv2

In [2]:
img = cv2.imread('Argazkiak/tomato.jpg')

In [3]:
hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
cv2.imshow('hsv irudia', hsv)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [4]:
yellowLow = (0, 80, 10)
yellowUp = (15, 255, 255)

In [5]:
mask = cv2.inRange(hsv, yellowLow, yellowUp)
cv2.imshow('maskaradun irudia', mask)
cv2.waitKey(0)
cv2.destroyAllWindows()

Behin azalpena emanda nola egingo den filtraketa, adibide bat egingo da tenis pelotearen inguraketarekin jarraituz. 

In [6]:
def read_rgb_image(image_name, show):
    rgb_image = cv2.imread(image_name)
    if show: 
        cv2.imshow("RGB Image",rgb_image)
    return rgb_image

In [7]:
def filter_color(rgb_image, lower_bound_color, upper_bound_color, show):
    #convert the image into the HSV color space
    hsv_image = cv2.cvtColor(rgb_image, cv2.COLOR_BGR2HSV)
    cv2.imshow("hsv image",hsv_image)

    #define a mask using the lower and upper bounds of the yellow color 
    mask = cv2.inRange(hsv_image, lower_bound_color, upper_bound_color)
    res = cv2.bitwise_and(rgb_image, rgb_image, mask=mask)
    if show: 
        cv2.imshow("Mask",mask)
        cv2.imshow("Result",res)

    return mask, res

In [8]:
def getContours(binary_image):      
    #_, contours, hierarchy = cv2.findContours(binary_image, 
    #                                          cv2.RETR_TREE, 
    #                                           cv2.CHAIN_APPROX_SIMPLE)
    contours, hierarchy = cv2.findContours(binary_image, 
                                            cv2.RETR_EXTERNAL,
	                                         cv2.CHAIN_APPROX_SIMPLE)
    return contours

In [9]:
def draw_ball_contour(binary_image, rgb_image, contours):
    black_image = np.zeros([binary_image.shape[0], binary_image.shape[1],3],'uint8')
    
    for c in contours:
        area = cv2.contourArea(c)
        perimeter= cv2.arcLength(c, True)
        ((x, y), radius) = cv2.minEnclosingCircle(c)
        if (area>100):
            cv2.drawContours(rgb_image, [c], -1, (150,250,150), 1)
            cv2.drawContours(black_image, [c], -1, (150,250,150), 1)
            cx, cy = get_contour_center(c)
            cv2.circle(rgb_image, (cx,cy),(int)(radius),(0,0,255),1)
            cv2.circle(black_image, (cx,cy),(int)(radius),(0,0,255),1)
            cv2.circle(black_image, (cx,cy),5,(150,150,255),-1)
            print ("Area: {}, Perimeter: {}".format(area, perimeter))
    print ("number of contours: {}".format(len(contours)))
    cv2.imshow("RGB Image Contours",rgb_image)
    cv2.imshow("Black Image Contours",black_image)



In [10]:
def get_contour_center(contour):
    M = cv2.moments(contour)
    cx=-1
    cy=-1
    if (M['m00']!=0):
        cx= int(M['m10']/M['m00'])
        cy= int(M['m01']/M['m00'])
    return cx, cy



In [11]:
def main():
    image_name = 'Argazkiak/tennisball01.jpg'
    yellowLower =(15, 60, 60)
    yellowUpper = (52, 255, 255)
    rgb_image = read_rgb_image(image_name, True)
    binary_image_mask, result = filter_color(rgb_image, yellowLower, yellowUpper, True)
    contours = getContours(binary_image_mask)
    draw_ball_contour(binary_image_mask, rgb_image,contours)

    cv2.waitKey(0)
    cv2.destroyAllWindows()



In [12]:
if __name__ == '__main__':
    main()

cv2.waitKey(0)
cv2.destroyAllWindows()

Area: 946.0, Perimeter: 130.71067714691162
number of contours: 10
